In [5]:
%pip install MCForecastTools

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement MCForecastTools (from versions: none)
ERROR: No matching distribution found for MCForecastTools


In [6]:
# Initial imports
import os
import csv
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import warnings
from pathlib import Path
import datetime as dt

#Subsequent imports
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
%matplotlib inline

In [7]:
def signal(pct_change):
    if pct_change>=0:
        return 1
    else:
        return 0

In [8]:
# List the ticker symbols for the stocks of interest
lst=['BMO.TO', 'BNS.TO' ]

# Determine the start and end dates
start_date = '2016-08-01'
end_date = '2022-07-31'

# Create empty dataframe to hold stock data
dfAllStocks=pd.DataFrame()


lstdf=[]

# Gathers stock data from listed tickers and combines them into a dataframe
for ticker in lst:
    # Create data frame from API in loop 
    df= yf.download(ticker, start= start_date, end= end_date)
    # Remove to '.TO' suffix from Canadian stocks
    tic= ticker.replace('.TO','')
    df['Ticker']=tic
   
    
    #Do all indicator calculations here
    df['ClosePct']=(df['Close']-df['Low'])/(df['High']-df['Low'])
    df['Range']=df['High']-df['Low']
    df['RangeMa']=df['Range'].rolling(30).mean()
    df['RangeSD']=df['Range'].rolling(30).std()
    df['RangeZscore']=(df['Range']-df['RangeMa'])/df['RangeSD']
    df['Y']=df['Close'].pct_change().shift(-1)
    df['Signal']=df['Y'].apply(signal)
    display(df)
    
    df=df.reset_index()
    df.drop(columns= ["Open", "High", "Low", "Volume", "Close",'Range','RangeSD','RangeMa','Adj Close','Y'], inplace= True)
   
    lstdf.append(df)
    del(df)
    
    
    

    
   
    # Combine individual stocks into a single data frame
dfAllStocks=pd.concat(lstdf, axis= 0)



# Drop the N/As
dfAllStocks = dfAllStocks.dropna()

# Drop the time component of the date
#dfAllStocks_Train.index = dfAllStocks_Train.index.date
dfAllStocks=dfAllStocks.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])#.sort_index('Date','Ticker')
# Display the data
display(dfAllStocks['2016-09-13':'2017-09-13'][:])

X=dfAllStocks.drop(columns=['Signal'])
Y=dfAllStocks['Signal']
Xtrain=X['2016-09-13':'2020-09-14'][:]
Xtest=X['2020-09-15':][:]
Ytrain=Y['2016-09-13':'2020-09-14'][:]
Ytest=Y['2020-09-15':][:]

display(Xtest)
display(Ytest)



[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Ticker,ClosePct,Range,RangeMa,RangeSD,RangeZscore,Y,Signal
Date,,,,,,,,,,,,,,
2016-08-02,83.199997,83.519997,82.379997,82.690002,60.470272,1125200,BMO,0.271935,1.139999,NaN,NaN,NaN,-0.003023,0
2016-08-03,82.779999,83.139999,82.230003,82.440002,60.287468,1247700,BMO,0.230769,0.909996,NaN,NaN,NaN,-0.004124,0
2016-08-04,82.339996,82.699997,81.620003,82.099998,60.038799,1038200,BMO,0.444443,1.079994,NaN,NaN,NaN,0.009135,1
2016-08-05,82.529999,83.029999,82.480003,82.849998,60.587292,930900,BMO,0.672724,0.549995,NaN,NaN,NaN,0.007725,1
2016-08-08,82.709999,83.669998,82.709999,83.489998,61.055325,878100,BMO,0.812500,0.959999,NaN,NaN,NaN,0.003713,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,125.000000,126.790001,125.000000,126.400002,116.456253,3681300,BMO,0.782123,1.790001,2.027335,0.654516,-0.362609,-0.004430,0
2022-07-26,126.129997,127.010002,125.419998,125.839996,115.940292,4900100,BMO,0.264149,1.590004,2.034335,0.648449,-0.685220,0.011682,1
2022-07-27,126.599998,127.940002,125.750000,127.309998,117.294655,5644700,BMO,0.712327,2.190002,2.023668,0.642964,0.258700,0.004320,1


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Ticker,ClosePct,Range,RangeMa,RangeSD,RangeZscore,Y,Signal
Date,,,,,,,,,,,,,,
2016-08-02,45.439983,45.536900,45.121544,45.460751,28.604271,1750300,BNS,0.816666,0.415356,NaN,NaN,NaN,-0.000761,0
2016-08-03,45.523056,45.820728,45.384602,45.426140,28.582481,1441400,BNS,0.095244,0.436127,NaN,NaN,NaN,-0.001676,0
2016-08-04,45.474594,45.613049,45.059238,45.349991,28.534565,987900,BNS,0.525004,0.553810,NaN,NaN,NaN,0.012822,1
2016-08-05,45.516132,45.945335,45.460751,45.931488,28.900465,1289700,BNS,0.971424,0.484585,NaN,NaN,NaN,0.004823,1
2016-08-08,45.876106,46.291466,45.862263,46.153011,29.039839,1305800,BNS,0.677415,0.429203,NaN,NaN,NaN,0.003450,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,68.418472,69.117546,68.291367,68.763466,61.136192,4541900,BNS,0.571425,0.826180,1.089163,0.428190,-0.614176,0.001584,1
2022-07-26,68.463867,68.999519,68.418472,68.872414,61.233063,5831900,BNS,0.781247,0.581047,1.072822,0.438135,-1.122427,0.011073,1
2022-07-27,69.026756,70.025436,68.935966,69.635040,61.911102,6234600,BNS,0.641664,1.089470,1.073730,0.438141,0.035925,0.006519,1


ClosePct  RangeZscore  Signal
Date       Ticker                               
2016-09-13 BMO     0.587301    -0.375516       1
           BNS     0.393618     1.854262       1
2016-09-14 BMO     0.407890     0.159795       1
           BNS     0.516483     1.556639       1
2016-09-15 BMO     0.666664     0.702966       0
...                     ...          ...     ...
2017-09-11 BNS     0.628574    -1.337678       1
2017-09-12 BMO     0.732760     0.524034       1
           BNS     0.732139    -0.468600       0
2017-09-13 BMO     0.625003    -0.791315       0
           BNS     0.365856    -1.027909       1

[504 rows x 3 columns]

ClosePct  RangeZscore
Date       Ticker                       
2020-09-15 BMO     0.318681    -0.534797
           BNS     0.541671    -1.265703
2020-09-16 BMO     0.036143    -0.623515
           BNS     0.088612    -0.068851
2020-09-17 BMO     0.718184    -0.236388
...                     ...          ...
2022-07-27 BNS     0.641664     0.035925
2022-07-28 BMO     0.874215    -0.626049
           BNS     0.847463     0.017401
2022-07-29 BMO     0.542171    -0.516795
           BNS     0.666669    -0.046215

[942 rows x 2 columns]

Date        Ticker
2020-09-15  BMO       0
            BNS       0
2020-09-16  BMO       1
            BNS       1
2020-09-17  BMO       0
                     ..
2022-07-27  BNS       1
2022-07-28  BMO       0
            BNS       1
2022-07-29  BMO       0
            BNS       0
Name: Signal, Length: 942, dtype: int64